<a href="https://colab.research.google.com/github/MSHQD/HWR/blob/main/Full_text_recognition_mmdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import sys
# print("version:", sys.version)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import albumentations as A
import random
import json
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
import torch, torchvision
import warnings
import sys
from skimage import io
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import shutil
from tqdm import tqdm
from matplotlib import pyplot as plt
import torch.optim as optim
import torchvision
import collections
from torchvision import datasets, models, transforms


In [ ]:
import torch
from torch import nn

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
import zipfile
zip1 = '/content/drive/MyDrive/train_recognition.zip'
extract_to1 = '/content/train_recognition'

with zipfile.ZipFile(zip1, 'r') as zip_ref:
    zip_ref.extractall(extract_to1)

zip2 = '/content/drive/MyDrive/train_segmentation.zip'
extract_to2 = '/content/train_segmentation'
with zipfile.ZipFile(zip2, 'r') as zip_ref:
    zip_ref.extractall(extract_to2)


In [ ]:
# data_dir = '/content/drive/MyDrive/НТИ ИИ /data_team/'
# project_dir = '/content/drive/MyDrive/НТИ ИИ /'
# data_dir = 'data'
project_dir = '/content'
data_dir = 'train_recognition'
train_images = 'images'
train_images = '/content/train_recognition/train_recognition/images'
image_size = 256
first_size = 512

# Settings

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10/index.html
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection && pip install -e . && python setup.py develop

In [ ]:
# !unzip -q '/content/drive/MyDrive/train_segmentation.zip'

In [ ]:
# !git clone https://bitbucket.org/william_rusnack/minimumboundingbox.git
# sys.path.append('minimumboundingbox')

In [ ]:
!pip install transformers -q
!pip install -q datasets jiwer
!pip install sentencepiece -q

# Building datasets

In [ ]:
# labels_path = os.path.join(project_dir, data_dir, 'labels.csv')
annotation = pd.read_csv('/content/train_recognition/train_recognition/labels.csv')
# annotation = pd.read_csv(labels_path)
annotation.head(10)

In [ ]:
image = plt.imread(os.path.join(train_images, '0.png'))
plt.imshow(image)
plt.show()

In [ ]:
len(os.listdir(train_images))

In [ ]:
import json

with open('/content/train_segmentation/train_segmentation/annotations.json') as f:
    seg_annotations = json.load(f)

In [ ]:
from sklearn.model_selection import train_test_split

val_annot = seg_annotations.copy()
train_annot = seg_annotations.copy()

train_annot['images'], val_annot['images']  = train_test_split(seg_annotations['images'], test_size=0.1, random_state=42)

In [ ]:
with open(os.path.join(data_dir, '/content/train_segmentation/train_segmentation/annotations_val.json'), 'w') as outfile:
    json.dump(val_annot, outfile)


with open(os.path.join(data_dir, '/content/train_segmentation/train_segmentationannotations_train.json'), 'w') as outfile:
    json.dump(train_annot, outfile)

# Train mask rcnn

## Train mask-rcnn

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_instance_segmentation_model(num_classes = 2):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
model = get_instance_segmentation_model()
model = model.to(device)


params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(params, lr=3e-4)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=20,
                                               gamma=0.5)

In [ ]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install -U openmim
!mim install "mmcv>=2.0.0rc4,<2.1.0"
!mim install "mmdet>=3.0.0,<3.2.0"

# Проверка
import torch, mmcv, mmdet
print("Torch:", torch.__version__)
print("MMCV:", mmcv.__version__)
print("MMDetection:", mmdet.__version__)

In [ ]:
from mmdet.datasets import build_dataset
from mmengine.dataset import build_dataloader

# Загрузка конфигурации
from mmengine.config import Config
cfg = Config.fromfile('configs/mask_rcnn/mask_rcnn_x101_32x8d_fpn_mstrain-poly_3x_coco.py')

# Настройка путей и классов
cfg.dataset_type = 'CocoDataset'
cfg.data_root = '/content/train_recognition/'
cfg.data.train.ann_file = 'labels.json'
cfg.data.train.img_prefix = 'images'
cfg.data.train.classes = ('text',)

# Создание датасета и загрузчика
datasets = [build_dataset(cfg.data.train)]
train_loader = build_dataloader(
    datasets[0],
    samples_per_gpu=2,
    workers_per_gpu=2,
    dist=False,
    shuffle=True
)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, num_epochs, model_name):
    model.to(device)
    history = {'train_loss': []}

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for images, targets in train_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            epoch_loss += losses.item()

        lr_scheduler.step()
        avg_loss = epoch_loss / len(train_loader)
        history['train_loss'].append(avg_loss)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # Сохраняем модель
    torch.save(model.state_dict(), f'{model_name}.pth')

    return history

In [ ]:
history = train_model(model, train_loader, val_loader, None, optimizer, lr_scheduler, 30, 'mask_rcnn_small_2')

# Generate poligons

### Get masks

In [ ]:
sys.path.append('mmdetection')

import mmdet
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmcv import Config
from mmdet.apis import init_detector, inference_detector, show_result_pyplot, set_random_seed

In [ ]:
class SegmentationModel:
  def __init__(self, model_path, cfg):
    with open('labels.txt', 'w') as f:
      f.write('text')
    self.cfg = cfg
    self.cfg.load_from = model_path
    self.model = init_detector(self.cfg, model_path, device=device)
    self.model.CLASSES = ['text']
    self.model.cfg = self.cfg
    self.threshold = 0.3

  def eval(self):
    return self

  def __call__(self, batch):
    result = mmdet.apis.inference.inference_detector(self.model, batch)
    output = []
    for img in result:
      out_img = []
      for box in img[0]:
        out = []
        #print(len(box))
        left, top, right, bottom, c = box
        if c > self.threshold:
          out.append([[left, top]])
          out.append([[right, top]])
          out.append([[right, bottom]])
          out.append([[left, bottom]])
          out_img.append(out)
      output.append(out_img)
    return np.array(output).astype(int)

In [ ]:
import config
importlib.reload(config)

seg_model = SegmentationModel('/content/drive/MyDrive/НТИ ИИ /team/semeka_models/epoch_21.pth',
                              config.cfg)

In [ ]:
image = cv2.imread('/content/train_segmentation/images/0_0_eng.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

contours = seg_model([image])

for contour in contours[0]:
    cv2.drawContours(image, np.array([contour.astype(int)]), -1, (0, 255, 0), 2)


plt.figure(figsize=(15,15))
plt.imshow(image)
plt.show()

# Loading TrOcr

In [ ]:
def plot_images(images_for_show):
  """Строит изображение на одном графике"""
  fig = plt.figure(figsize=(16, 16))

  columns = len(images_for_show)
  rows = 1
  for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(np.clip(images_for_show[i - 1], 0, 1))

  fig.subplots_adjust(wspace=0.1, hspace=0)
  plt.show()

In [ ]:
class AlbuPadding(A.DualTransform):
    def __init__(self, always_apply=False, p=0.5):
        super(AlbuPadding, self).__init__(always_apply, p)

    def apply(self, image, **params):
        zeros = np.zeros((128, 384, 3))
        image = np.concatenate([zeros, image, zeros], axis=0)
        return image.astype(np.uint8)

In [ ]:
from albumentations.pytorch.transforms import ToTensor
from transformers import AutoFeatureExtractor, XLMRobertaTokenizer, VisionEncoderDecoderModel, RobertaTokenizer

class TrOcrModel:
  def __init__(self, model_path, padding=True):
      self.model = VisionEncoderDecoderModel.from_pretrained(model_path).to(device)
      self.model.eval()

      self.feature_extractor = AutoFeatureExtractor.from_pretrained('microsoft/trocr-small-handwritten')
      self.tokenizer = XLMRobertaTokenizer.from_pretrained('microsoft/trocr-small-handwritten')

      if padding:
        self.transforms = A.Compose([
                A.Resize(128, 384),
                AlbuPadding(always_apply=True),
            ])
      else:
        self.transforms = A.Compose([
                A.Resize(384, 384),
            ])

  def image_preprocess(self, image):
      image = self.transforms(image=image)['image']
      pixel_values = self.feature_extractor(image, return_tensors="pt").pixel_values
      return pixel_values

  def predict_batch(self, images):
      batch = torch.concat([self.image_preprocess(image) for image in images], axis=0).to(device)
      outputs = self.model.generate(batch)
      return [self.tokenizer.decode(pred.cpu().numpy(), skip_special_tokens=True) for pred in outputs]

  def __call__(self, image):
      pred = self.model.generate(self.image_preprocess(image).to(device))
      return self.tokenizer.decode(pred[0].cpu().numpy(), skip_special_tokens=True)

In [ ]:
text_model = TrOcrModel("/content/drive/MyDrive/НТИ ИИ /team/sergey_models/tr_ocr_best_small_aug_nti2data")

In [ ]:
image = cv2.imread(os.path.join(train_images, '0.png'))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
text_model(image)

In [ ]:
text_model.predict_batch([image, image])

# Making predict

In [ ]:
from PIL import ImageFont, ImageDraw, Image

In [ ]:
def crop_img_by_polygon(img, polygon):
    # https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python
    pts = np.array(polygon)
    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    croped = img[y:y+h, x:x+w]
    pts = pts - pts.min(axis=0)
    mask = np.zeros(croped.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
    dst = cv2.bitwise_and(croped, croped, mask=mask)
    return dst

In [ ]:
def get_image_visualization(img, pred_data, fontpath, font_koef=50):
    img = img.copy()
    h, w = img.shape[:2]
    font = ImageFont.truetype(fontpath, int(h/font_koef))
    empty_img = Image.new('RGB', (w, h), (255, 255, 255))
    draw = ImageDraw.Draw(empty_img)

    for prediction in pred_data['predictions']:
        polygon = prediction['polygon']
        pred_text = prediction['text']
        cv2.drawContours(img, np.array([polygon]), -1, (0, 255, 0), 2)
        x, y, w, h = cv2.boundingRect(np.array([polygon]))
        draw.text((x, y), pred_text, fill=0, font=font)
        # print(pred_text, x, y)

    vis_img = np.array(empty_img)
    vis = np.concatenate((img, vis_img), axis=1)
    return vis

In [ ]:
def get_polygon_for_answer(polygon, croped):
    pts = np.array(polygon)
    rect = cv2.boundingRect(pts)
    x1,y1,w,h = rect
    mid_x = x1 + w // 2
    mid_y = y1 + h // 2

    best = 1e9
    for i in range(h):
      now = abs((croped[:i, :] != [0, 0, 0]).sum() - (croped[i:, :] != [0, 0, 0]).sum())
      if now < best:
        best = now
        mid_y = i + y1

    x1 = mid_x - w // 5
    x2 = mid_x + w // 5
    # return [(mid_y, x1), (mid_y + 5, x1), (mid_y + 5, x2), (mid_y, x2)]
    return [(x1, mid_y), (x1, mid_y + 1), (x2, mid_y + 1), (x2, mid_y)]

In [ ]:
def get_classifier_model():

  model = models.resnet50(pretrained=True)
  model.fc = nn.Sequential(
      nn.Linear(2048, 256),
      nn.ReLU(),
      nn.Linear(256, 2)
  )
  return model

In [ ]:
with open(os.path.join(data_dir, 'train_segmentation/annotations.json'), 'r') as f:
    annotations = json.load(f)

def get_image_cer(image_name):
  img = cv2.imread('data/train_segmentation/images/' + image_name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  pred_data = {}
  pred_data[image_name] = pipeline_model(img)
  with open('prediction.json', "w") as f:
    json.dump(pred_data, f)

  now = annotations.copy()
  for el in annotations['images']:
    if el['file_name'] == image_name:
      now['images'] = [el]
      break


  with open(os.path.join('annotations_now.json'), 'w') as outfile:
      json.dump(now, outfile)
  print('predict is ready')

  print(os.popen('python3 data/evaluate.py --ref_path annotations_now.json --pred_path prediction.json').read())

In [ ]:
np.argsort([0, 3, 2])

In [ ]:
import config

class PiepleinePredictor:
    def __init__(self, segm_model_path, ru_ocr_model_path, en_ocr_model_path, classifier_model_path):
        # self.seg_model = SEGMpredictor(segm_model_path)
        self.seg_model = SegmentationModel(segm_model_path, config.cfg)

        self.text_model = {'ru': TrOcrModel(ru_ocr_model_path),
                           'en': TrOcrModel(en_ocr_model_path)}
        self.batch_size = 8
        self.transforms = A.RandomScale(scale_limit=(-0.5, -0.5), p=1)

        self.classifier_model = torch.load(classifier_model_path, map_location=device)
        self.classifier_predicts = 25
        self.classifier_transforms = A.Compose([
                  A.Resize(128, 384),
                  ToTensor()
              ])

    def predict_language(self, images):
        p = np.argsort([el.shape[1] for el in images])
        batch = []
        for idx in p[-self.classifier_predicts:]:
          batch.append(self.classifier_transforms(image=images[idx])['image'])

        batch = torch.stack(batch).to(device)
        preds = self.classifier_model(batch)
        preds = torch.argmax(preds, dim=1)
        # plot_images([torch.moveaxis(el, 0, -1).detach().cpu().numpy() for el in batch[:10]])
        # print(preds[:10])
        res = preds.sum() > len(preds) / 2
        return 'ru' if res else 'en'

    def __call__(self, img, return_only_language=False):
        img = img.copy()
        # img = self.transforms(image=img)['image']
        with torch.no_grad():
          output = {'predictions': []}
          bgr = cv2.cvtColor(img.copy(), cv2.COLOR_RGB2BGR)
          contours = self.seg_model([bgr])[0]
          images = []
          not_none_contours = []
          for contour in contours:
              if contour is not None:
                  crop = crop_img_by_polygon(img, contour)
                  images.append(crop)
                  not_none_contours.append(contour)

          language = self.predict_language(images)
          if return_only_language:
            return language

          predicted_text = []
          for i in range(0, len(images), self.batch_size):
            predicted_text += self.text_model[language].predict_batch(images[i:i + self.batch_size])


          # for img, text in zip(images[:30], predicted_text[:30]):
          #   plt.imshow(img)
          #   plt.title(text)
          #   plt.show()

          for pred_text, contour, crop in zip(predicted_text, not_none_contours, images):
            output['predictions'].append({
                              # 'polygon': [[int(i[0][0] * 2), int(i[0][1] * 2)] for i in contour],
                              # 'polygon': [[int(i[0][0]), int(i[0][1])] for i in contour],
                              'polygon': get_polygon_for_answer(contour, crop),
                              'text': pred_text
                            })
        return output

In [ ]:
pipeline_model = PiepleinePredictor(
                                    '/content/drive/MyDrive/НТИ ИИ /team/semeka_models/epoch_21.pth',
                                    # '/content/drive/MyDrive/НТИ ИИ /team/sergey_models/mask_rcnn_small',
                                    "/content/drive/MyDrive/НТИ ИИ /team/sergey_models/tr_ocr_best_small",
                                    "/content/drive/MyDrive/НТИ ИИ /team/sergey_models/tr_ocr_best_eng_padding",
                                    "/content/drive/MyDrive/НТИ ИИ /team/sergey_models/language_classifier"
                                    ) #_aug_nti2data

In [ ]:
get_image_cer('0_0_eng.jpg')

In [ ]:
get_image_cer('100_0.JPG')

In [ ]:
get_image_cer('6_1_eng.jpg')

In [ ]:
path = 'data/train_segmentation/images'

sum = 0
right = 0
for image_name in tqdm(os.listdir(path)[:]):
  real = 'en' if 'eng' in image_name else 'ru'
  img = cv2.imread(os.path.join(path, image_name))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  pred = pipeline_model(img, True)
  sum += 1
  right += pred == real
  if pred != real:
    print(image_name, real)

print()
print(right, sum)

In [ ]:
img = cv2.imread('data/train_segmentation/images/105_0.JPG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
output = pipeline_model(img)
vis = get_image_visualization(img, output, os.path.join(data_dir, 'font.otf'))

plt.figure(figsize=(20, 20))
plt.imshow(vis)
plt.show()

In [ ]:
img = cv2.imread('data/train_segmentation/images/105_0.JPG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = A.RandomScale(scale_limit=(-0.5, -0.5), p=1)(image=img)['image']

plt.figure(figsize=(15, 15))
plt.imshow(img)

# Save predictions from dataset

In [ ]:
# Путь к json'у с разметкой
DATA_JSON_PATH = 'data/train_segmentation/annotations_val.json'

# Папка с картинками тетрадей
IMAGE_ROOT = 'data/train_segmentation/images/'

# Файл для сохранения предсказаний пайплайна. Один json
SAVE_PATH = os.path.join(project_dir, 'prediction.json')

In [ ]:
with open(DATA_JSON_PATH, 'r') as f:
    data = json.load(f)

pred_data = {}

In [ ]:
for data_img in tqdm(data['images']):
    img_name = data_img['file_name']
    image = cv2.imread(os.path.join(IMAGE_ROOT, img_name))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pred_data[img_name] = pipeline_model(image)

In [ ]:
for data_img in tqdm(data['images']):
    img_name = data_img['file_name']
    image = cv2.imread(os.path.join(IMAGE_ROOT, img_name))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pred_data[img_name] = pipeline_model(image)

In [ ]:
with open('prediction.json', "w") as f:
    json.dump(pred_data, f)

In [ ]:
#small
!python3 data/evaluate.py --ref_path data/train_segmentation/annotations_val.json --pred_path prediction.json

In [ ]:
!python3 data/evaluate.py --ref_path data/train_segmentation/annotations_val.json --pred_path prediction.json

In [ ]:
# mask_rcnn_small + en_ocr_padding + classifier not regression
!python3 data/evaluate.py --ref_path data/train_segmentation/annotations_val.json --pred_path prediction.json

In [ ]:
# mask_rcnn_small + en_ocr_padding + classifier regression
!python3 data/evaluate.py --ref_path data/train_segmentation/annotations_val.json --pred_path prediction.json

In [ ]:
# mask_rcnn_small + en_ocr_padding + classifier not regression
!python3 data/evaluate.py --ref_path data/train_segmentation/annotations_val.json --pred_path prediction.json

In [ ]:
# detectoRS + en_ocr + classifier not regression
!python3 data/evaluate.py --ref_path data/train_segmentation/annotations_val.json --pred_path prediction.json